<a href="https://colab.research.google.com/github/mxbb-0-1/20MIS1068_AI/blob/main/Model%20Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Name: Anselm Sherwin W
###Regno: 20MIS1068



###Download the File

In [1]:
import json
import pandas as pd

with open('idmanual.json', 'r') as file:
    data = json.load(file)

# Convert the JSON data to a pandas DataFrame
df = pd.DataFrame(data)

# Extract necessary columns
descriptions = df['description'].values
statuses = df['status'].map({'A': 1, 'D': 0}).values
class_ids = df['class_id'].values


### Tokenization and Text Cleaning

In [2]:
# Summary statistics
print(df.describe())

# Check for missing values
print(df.isnull().sum())
# Drop rows with missing values
df = df.dropna()

# Alternatively, fill missing values with a placeholder
df['description'] = df['description'].fillna('Unknown')


          id_tx class_id   description status
count     58702    58702         58702  58702
unique    58700       49         58602      4
top     016-471      009  Paper blinds      A
freq          2     5457             2  49477
id_tx          0
class_id       0
description    0
status         0
dtype: int64


In [3]:
import re

def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and digits
    text = re.sub(r'[^a-z\s]', '', text)
    return text

df['description'] = df['description'].apply(clean_text)


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['class_id_encoded'] = label_encoder.fit_transform(df['class_id'])
df['status_encoded'] = df['status'].map({'A': 1, 'D': 0})


In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the descriptions
inputs = tokenizer(df['description'].tolist(), padding=True, truncation=True, return_tensors='pt')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
import torch

# Prepare tensors for labels and status
labels = torch.tensor(df['class_id_encoded'].values)
status_tensor = torch.tensor(df['status_encoded'].values).unsqueeze(1)

# inputs['input_ids'], inputs['attention_mask'] are ready for model input


In [7]:
!pip install wandb
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification
import wandb

# Initialize WandB for logging
wandb.init(project="trademark-class-prediction")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
# Continue from where Step 1 left off
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the descriptions
inputs = tokenizer(df['description'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Prepare tensors for labels and status
labels = torch.tensor(df['class_id_encoded'].values)
status_tensor = torch.tensor(df['status_encoded'].values).unsqueeze(1)

# Combine input tensors into a dataset
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)


In [9]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader for batch processing
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [10]:
# Load pre-trained BERT model with a classification head
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['class_id'].unique()))

# Use GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Training parameters
num_epochs = 3
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Compute loss
        loss = criterion(logits, labels)
        total_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Compute accuracy
        predictions = torch.argmax(logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions

    # Log metrics to WandB
    wandb.log({
        "epoch": epoch + 1,
        "loss": avg_loss,
        "accuracy": accuracy
    })

    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

# Save the model
torch.save(model.state_dict(), "trademark_classification_model.pth")



Epoch 1/3 - Loss: 1.5573, Accuracy: 0.6245
Epoch 2/3 - Loss: 0.5415, Accuracy: 0.8593
Epoch 3/3 - Loss: 0.3172, Accuracy: 0.9157


In [12]:
# Evaluation on the validation set
model.eval()
correct_predictions = 0
total_predictions = 0
total_val_loss = 0

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        loss = criterion(logits, labels)
        total_val_loss += loss.item()

        predictions = torch.argmax(logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

avg_val_loss = total_val_loss / len(val_loader)
val_accuracy = correct_predictions / total_predictions

# Log validation metrics to WandB
wandb.log({
    "val_loss": avg_val_loss,
    "val_accuracy": val_accuracy
})

print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Validation Loss: 0.3904, Validation Accuracy: 0.8932
